# LM-STUDIO, HUGGINGFACE 선택할 수 있는 radio button 추가
# + enter키로 요청 전송 + "종료" 입력시 프로그램 종료

In [ ]:
# radio button 을 통해 api 이용 방식 select 가능
# 챗봇에게 요청할 내용 입력 후 enter를 누르면 요청 전송 가능
# 입력란에 "종료" 입력시 프로그램 종료
import torch
from transformers import pipeline
from PyQt5.QtWidgets import *
from openai import OpenAI
import keyboard

import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device=device, torch_dtype=torch.bfloat16)

def qa_hugging(context):
    data = [
        [
            {
                "role": "system",
                "content": [{"type": "text", "text": "너는 챗봇이야."},]
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": context},]
            },
        ],
    ]
    
    return data

def qa_lm(req_data):
    url = "http://localhost:1234/v1"
    client = OpenAI(base_url=url, api_key='lm-studio')
    
    completion = client.chat.completions.create(
        model = 'model-identifier',
        messages = [
            {"role": "user", "content": req_data}
        ],
        # 아래의 조정값들이 한글에서는 잘 안먹는것 같다
        temperature = 0.1,
        max_tokens = 1024,
        top_p = 1,
        frequency_penalty = 0,
        presence_penalty = 0,
        stop = ["..."] # 생성된 텍스트에서 종료 구문을 설정
    )
    model_answer = completion.choices[0].message.content

    return model_answer


class ChatApp(QWidget):
    def __init__(self):
        global platform
        super().__init__()
        self.setWindowTitle("AI금융소프트웨어 LLM chat bot")
        self.setGeometry(100, 100, 1000, 1000)

        # 레이아웃 설정
        layout = QVBoxLayout()

        # 상단 라벨
        self.label = QLabel("홍필두 교수님 너무 잘생기셨어!! 최고야!!", self)
        self.label.setStyleSheet("background-color: lightgreen")
        font = self.label.font()
        font.setPointSize(20)
        self.label.setFont(font)
        layout.addWidget(self.label)

        self.radio1 = QRadioButton(self)
        font = self.label.font()
        font.setPointSize(15)
        self.label.setFont(font)
        self.radio1.setText("huggingface 이용하기(주의: 느림)")
        self.radio1.toggled.connect(self.switch_platform_to_huggingface)
        # if self.radio1.isChecked() == True:
        #     self.switch_platform_to_huggingface()
        # print("radio1 checked:", self.radio1.isChecked())
        # print("platform:", platform)
        layout.addWidget(self.radio1)
        
        self.radio2 = QRadioButton(self)
        font = self.label.font()
        font.setPointSize(15)
        self.label.setFont(font)
        self.radio2.setText("lm-studio 이용하기")
        self.radio2.toggled.connect(self.switch_platform_to_lm_studio)
        # if self.radio2.isChecked() == True:
        #     self.switch_platform_to_lm_studio()
        # print("radio2 checked:", self.radio2.isChecked())
        layout.addWidget(self.radio2)

        # 채팅 내용 표시창
        self.chat_box = QTextEdit(self)

        font = self.chat_box.font()
        font.setPointSize(20)
        self.chat_box.setFont(font)

        self.chat_box.setReadOnly(True) # 채팅 내용 출력 부분은 Read Only
        layout.addWidget(self.chat_box)

        # 사용자 입력 필드
        self.user_input = QLineEdit(self)

        font = self.user_input.font()
        font.setPointSize(20)
        self.user_input.setFont(font)

        self.user_input.setPlaceholderText("여기에 챗봇에게 요청할 내용을 입력하세요.")
        layout.addWidget(self.user_input)

        # 전송 버튼
        self.send_button = QPushButton("보내기", self)

        font = self.send_button.font()
        font.setPointSize(20)
        self.send_button.setFont(font)

        keyboard.add_hotkey('enter', lambda:self.send_message())
        self.send_button.clicked.connect(self.send_message) # 이 버튼을 클릭시 send_message 함수 수행
        layout.addWidget(self.send_button)

        # 레이아웃을 윈도우에 설정
        self.setLayout(layout)
        
    def switch_platform_to_huggingface(self, checked):
        global platform
        if checked:
            platform = "huggingface"
    def switch_platform_to_lm_studio(self, checked):
        global platform
        if checked:
            platform = "lm-studio"

    def send_message(self):
        # 사용자 메시지 가져오기
        user_message = self.user_input.text()
        global platform
        print("send_message를 할때의 platform:", platform)
        if user_message == "종료":
            self.exit_()
            print("▶ qt5를 종료합니다")
        
        if user_message != "종료":
            self.chat_box.append(f'질문: {user_message}') # 질문을 채팅창에 써주고
            self.user_input.clear() # 질문 입력창은 지우고
            #########################################################################
            # 여기부터 수정 시작해야함(라디오버튼 에따른 함수 호출을 달리 해야함)
            # 모델에 요청을 보내고 응답 받기
            if platform == "lm-studio":
                bot_message = qa_lm(f"{user_message}")
                self.chat_box.append(f"LM-STUDIO: {bot_message}") # 챗봇의 응답을 채팅창에 출력
                self.chat_box.append("-" * 50)
            elif platform == "huggingface":
            # 모델에 요청을 보내고 응답 받기
                messages = qa_hugging(f"{user_message}")
                output = pipe(messages, max_new_tokens=1024)
                bot_message = output[0][0]["generated_text"][2]["content"]                
                self.chat_box.append(f"HUGGINGFACE: {bot_message}") # 챗봇의 응답을 채팅창에 출력
                self.chat_box.append("-" * 50)
            else:
                self.chat_box.append("챗봇을 이용하려면 AI 플랫폼을 먼저 선택하세요")
                self.chat_box.append("-" * 50)

    def exit_(self):
        self.close() #끝나면 qt5 종료

import sys
platform = "platform not selected"
app = QApplication(sys.argv) # qt5 시작
window = ChatApp() # 내가만든 메소드 시작
window.show() # 화면 띄우기
sys.exit(app.exec_()) # 프로그램 끝 리소스 반환